# Load Libraries
Load libraries and initialize the template for a translation task

In [2]:
from dotenv import load_dotenv
load_dotenv()

from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue

template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage(
            "Translate the following text to {{?to_lang}}: {{?text}}"
        ),
    ],
    defaults=[
        TemplateValue(name="to_lang", value="German"),
    ],
)

This template can be used to create translation requests where the language and text to be translated are specified dynamically. The placeholders in the `UserMessage` will be replaced with the actual values provided at runtime, and the default value for the language is set to German.

# Define the LLM 
The `LLM` class is used to configure and initialize a language model for generating text based on specific parameters. In this example, we'll use the gpt-4o model to perform the translation task.

In [3]:
from gen_ai_hub.orchestration.models.llm import LLM

llm = LLM(name="gpt-4o", version="latest", parameters={"max_tokens": 256, "temperature": 0.2})

# Create an Orchestration Configuration
The `OrchestrationConfig` class defines a configuration for integrating various modules, such as templates and language models, into a cohesive orchestration setup.

In [4]:
from gen_ai_hub.orchestration.models.config import OrchestrationConfig

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

# Run the Orchestration Request
The `OrchestrationService` class is used to interact with a orchestration service instance by providing configuration details to initiate and manage its operations.

In [5]:
from gen_ai_hub.orchestration.service import OrchestrationService

orchestration_service = OrchestrationService(config=config)
result = orchestration_service.run(template_values=[
    TemplateValue(name="text", value="The Orchestration Service is working!")
])
print(result.orchestration_result.choices[0].message.content)

Der Orchestrierungsdienst funktioniert!


# Different `response_format` Parameter Options
The `response_format` parameter in `Template` allows the model output to be formatted in several predefined ways, as follows:
1. **text**: This is the simplest form where the model's output is generated as plain text. It is suitable for applications that require raw text processing.

2. **json_object**: Under this setting, the model's output is structured as a JSON object. This is useful for applications that handle data in JSON format, enabling easy integration with web applications and APIs.

3. **json_schema**: This setting allows the model's output to adhere to a defined JSON schema. This is particularly useful for applications that require strict data validation, ensuring the output matches a predefined schema.

In [8]:
# Using text as response format
template = Template(
    messages=[
        SystemMessage("You are a helpful assistant."),
        UserMessage("{{?user_query}}")
    ],
    response_format="text",
    defaults=[
        TemplateValue(name="user_query", value="Who was the first person on the moon?")
    ]
)

# Recreate the config with the new template
config = OrchestrationConfig(
    template=template,
    llm=llm,
)

# Call the orchestration service with the new config
orchestration_service = OrchestrationService(config=config)
result = orchestration_service.run()

print(result.orchestration_result.choices[0].message.content)

The first person to set foot on the moon was astronaut Neil Armstrong. He did so on July 20, 1969, during the Apollo 11 mission.


In [9]:
# Using json_object as response format
template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage("{{?user_query}}")
    ],
    response_format="json_object",
    defaults=[
        TemplateValue(name="user_query", value="Who was the first person on the moon? in json")
    ]
)

# Recreate the config with the new template
config = OrchestrationConfig(
    template=template,
    llm=llm,
)

# Call the orchestration service with the new config
orchestration_service = OrchestrationService(config=config)
result = orchestration_service.run()

print(result.orchestration_result.choices[0].message.content)

{
  "name": "Neil Armstrong",
  "date": "July 20, 1969",
  "mission": "Apollo 11",
  "nationality": "American"
}


**Important**: When using response_format as json_object, ensure that messages contain the word 'json' in some form

In [10]:
# Using json_schema as response format
from gen_ai_hub.orchestration.models.response_format import ResponseFormatJsonSchema 

json_schema = {
    "title": "Person",
    "type": "object",
    "properties": {
            "firstName": {
            "type": "string",
            "description": "The person's first name."
        },
            "lastName": {
            "type": "string",
            "description": "The person's last name."
        }
    }
}
template = Template(
    messages=[
        SystemMessage("You are a helpful translation assistant."),
        UserMessage("{{?user_query}}")
    ],
    response_format = ResponseFormatJsonSchema(
        name="person", 
        description="person mapping", 
        schema=json_schema),
    defaults=[
        TemplateValue(
            name="user_query", 
            value="Who was the first person on the moon?")
    ]
)

# Recreate the config with the new template
config = OrchestrationConfig(
    template=template,
    llm=llm,
)

# Call the orchestration service with the new config
orchestration_service = OrchestrationService(config=config)
result = orchestration_service.run()

print(result.orchestration_result.choices[0].message.content)

{"firstName":"Neil","lastName":"Armstrong"}
